In [0]:
#Librerias que se usaran
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales
import matplotlib.pyplot as plt # Libreria para realizar graficos 
from tabulate import tabulate   # Permite formatear y mostrar de mejor manera los datos tabulares
import seaborn as sns  # Libreria para realizar graficos y vizualizaciones
import psycopg2        # Libreria que permite la conexion con PostgresSQL
from matplotlib.backends.backend_pdf import PdfPages # Libreria que permite exportar graficos en pdf

#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

## 1. Extraemos los datos del C4M

In [0]:
# Información de la conexión a PostgreSQL
host = "srv-postgres-d4m.postgres.database.azure.com"
database = "ControlSenseDB"
user = "administrador"
password = "Protobuffers2024"

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Crear un cursor para ejecutar comandos SQL
    cursor = connection.cursor()

    # Establecer la zona horaria antes de ejecutar la consulta
    time_zone_query = "SET TIME ZONE 'America/Lima';"
    cursor.execute(time_zone_query)

    # Tu consulta SQL equipos de acarreo a considerar en c4mequipos y datacamion: 61,63,64,65,68,69,71,74,103,122,125,126,198,199,201,204,205
    # equipos acarreo disponibles en telemetria h4m(2dias) : 61,65,68,126,199,201,*204*,205
    # Query de Join entre DATACAMION Y TSEQUIPOS
    tu_query_sql = '''
    select main.*, e.id_equipo, e.nombre from (SELECT *
    FROM public.getsensorsvaluesmod(201, '2024-02-27 07:00:30.612-05', '2024-03-03 06:27:30.612-05') a --data_camion
    left join tp_tramosstat b
    on b.id = (select id from tp_tramosstat where id_tptramosstat = a.tramosidsnew_t and tiem_creac < a.instant_date_t
                    order by tiem_creac desc limit 1)) main
    left join ts_equipos e
    on main.eq_id = e.id_equipo				
    where main.nombre_tramo is not null
    order by main.instant_date_t asc
    '''
    # Ejecutar la consulta
    cursor.execute(tu_query_sql)

    # Obtener los resultados en un DataFrame de pandas
    resultados17 = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

    # Hacer lo que necesites con los resultados
    print(resultados17.head())

except psycopg2.Error as e:
    print("Error al conectar a la base de datos PostgreSQL:", e)

In [0]:
resultados17.head()

In [0]:
resultados17.shape

#### 1.1 Guardarmos directamente el dfpandas en un csv en el BlobStorage
Nota importante: Debes comentar la variable "connection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# 1.Guardamos el df convertido en csv en la variable csv_data
csv_data = resultados17.to_csv(index=False)

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_equipo201.csv"

# Subir el archivo CSV al Blob Storage
# with open("datos_raw_shougang_equipo61.csv", "rb") as data:
#     blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
#     blob_client.upload_blob(data)

# 5. Guardamos el archivo CSV al Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
blob_client.upload_blob(csv_data)

### 1.2 Unir todos los csv(datalake-BlobStorage), Se leen varios dfs(csv) se Unen en 1 solo
Nota importante: Debes comentar la variable "conection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from io import BytesIO
from azure.storage.blob import BlobServiceClient

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name)
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"

# 5. Se crea una lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# 5.1 Nombres de los archivos CSV almacenados en el Contenedor de BlobStorage
csv_files = ["datos_raw_shougang_equipo61.csv","datos_raw_shougang_equipo199.csv", "datos_raw_shougang_equipo201.csv"]

# # Leer cada archivo CSV y guardarlos en DataFrames individuales
# for csv_file in csv_files:
#     # Descargar el archivo CSV del Blob Storage
#     with open(csv_file, "wb") as my_blob:
#         blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file)
#         blob_data = blob_client.download_blob()
#         blob_data.readinto(my_blob)
    
#     # Leer el archivo CSV en un DataFrame de Pandas
#     df = pd.read_csv(csv_file)
    
#     # Agregar el DataFrame a la lista
#     dfs.append(df)

# 6. Leemos cada archivo CSV y guardarlos en DataFrames individuales
for csv_file in csv_files:
    # Obtener el blob del Blob Storage
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file)
    
    # Descargar el contenido del blob
    blob_data = blob_client.download_blob()
    
    # Leer el contenido del blob en un objeto BytesIO
    bytes_io = BytesIO()
    blob_data.download_to_stream(bytes_io)
    bytes_io.seek(0)  # Asegurar que la posición del cursor esté al inicio del archivo
    
    # Leer el archivo CSV en un DataFrame de Pandas desde el objeto BytesIO
    df = pd.read_csv(bytes_io)
    
    # Agregar el DataFrame a la lista
    dfs.append(df)

# 7. Se Concatena todos los DataFrames en uno solo df
result_df = pd.concat(dfs, ignore_index=True)

# Visualizar el DataFrame resultante
result_df.head()

In [0]:
result_df.shape

In [0]:
result_df['nombre'].unique()

#### 1.3 Guardar el csv consolidado C4M en el BlobStorage 
Nota importante: Debes comentar la variable "conection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient

# 1.Guardamos el df convertido en csv en la variable csv_data
datos_total = result_df.to_csv(index=False)

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_c4m.csv"

# 5. Guardar el archivo CSV al Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
blob_client.upload_blob(datos_total)

#### 1.4 Leer csv consolidado c4m desde el BlobStorage
Nota importante: Debes comentar la variable "conection_string" Si NO deja hacer COMMIT DEL #CODIGO EN GIT,GITHUB

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_c4m.csv"

# 5. Obtener el blob_client
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# 6. Leer el contenido del blob como texto
blob_data = blob_client.download_blob().content_as_text()

# 7. Leer el archivo CSV en un DataFrame de Pandas desde el texto
datos_c4m = pd.read_csv(io.StringIO(blob_data))
datos_c4m.head()

In [0]:
datos_c4m.shape

In [0]:
datos_c4m["eq_id"].unique()

In [0]:
datos_c4m['id_equipo'].unique()

In [0]:
datos_c4m['nombre'].unique()

## 2. Cargamos datos del H4M alojados en el BlobStorage

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from io import BytesIO
from azure.storage.blob import BlobServiceClient

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificar el Nombre del contenedor en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_h4m/operacion_shougang/equipo_fc98/"

# 5. Lista para almacenar los DataFrames de cada archivo CSV
dfs = []

# 5.1 Nombres de los archivos CSV
csv_files = ["fc98_allvar_00_7_1marzo.csv", "fc98_allvar_7_13_1marzo.csv","fc98_allvar_13_19_1marzo.csv",
             "fc98_allvar_19_2359_1marzo.csv","fc98_allvar_00_2359_2marzo.csv"]

# 6. Leer cada archivo CSV y guardarlos en DataFrames individuales
for csv_file in csv_files:
    # Obtener el blob del Blob Storage
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=csv_file)
    
    # Descargar el contenido del blob
    blob_data = blob_client.download_blob()
    
    # Leer el contenido del blob en un objeto BytesIO
    bytes_io = BytesIO()
    blob_data.download_to_stream(bytes_io)
    bytes_io.seek(0)  # Asegurar que la posición del cursor esté al inicio del archivo
    
    # Leer el archivo CSV en un DataFrame de Pandas desde el objeto BytesIO
    df = pd.read_csv(bytes_io)
    
    # Agregar el DataFrame a la lista
    dfs.append(df)

# 7. Concatenar todos los DataFrames en uno solo
result_h4m_fc98 = pd.concat(dfs, ignore_index=True)

# 8. Visualizar el DataFrame resultante
result_h4m_fc98.head()

In [0]:
result_h4m_fc89.shape, result_h4m_fc98.shape

## 2.1 Se unen los df de los equipos extraidos

In [0]:
# Unir los dos DataFrames por filas
resultado_h4m_allequipos = pd.concat([result_h4m_fc89, result_h4m_fc98])

In [0]:
resultado_h4m_allequipos.tail()

In [0]:
resultado_h4m_allequipos['Equipment'].unique()

### 2.1 Extraer datos de TP_EQUIPOS para extraer el Id_equipo y cruzar con H4M

In [0]:
# Información de la conexión a PostgreSQL
host = "srv-postgres-d4m.postgres.database.azure.com"
database = "ControlSenseDB"
user = "administrador"
password = "Protobuffers2024"

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Crear un cursor para ejecutar comandos SQL
    cursor = connection.cursor()

    # Establecer la zona horaria antes de ejecutar la consulta
    time_zone_query = "SET TIME ZONE 'America/Lima';"
    cursor.execute(time_zone_query)

    # Tu consulta SQL equipos de acarreo a considerar en c4mequipos y datacamion: 61,63,64,65,68,69,71,74,103,122,125,126,198,199,201,204,205
    # equipos acarreo disponibles en telemetria h4m(2dias) : 61,65,68,126,199,201,*204*,205
    tu_query_sql = '''
    select * from ts_equipos
    --where nombre = 'FC89'
    '''
    # Ejecutar la consulta
    cursor.execute(tu_query_sql)

    # Obtener los resultados en un DataFrame de pandas
    resultados_equipos = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

    # Hacer lo que necesites con los resultados
    print(resultados_equipos.head())

except psycopg2.Error as e:
    print("Error al conectar a la base de datos PostgreSQL:", e)

In [0]:
resultados_equipos.head()

### 2.2 Datos de H4M join TSEQUIPOS(C4M)

In [0]:
# Realizar left join 
result_h4m = pd.merge(resultado_h4m_allequipos, resultados_equipos[['nombre','id_equipo']], left_on='Equipment', right_on='nombre', how='left')
result_h4m.head()

In [0]:
result_h4m.shape

## 3. Preprocesamiento de Datos (Formato de Fechas)

In [0]:
datos_c4m.head()

In [0]:
datos_c4m.dtypes

In [0]:
result_h4m.head()

In [0]:
result_h4m.dtypes

### 4.1. Cambiamos a formato DateTime (Formato de Fechas)

In [0]:
# datos_c4m['instant_date_t'] = pd.to_datetime(datos_c4m['instant_date_t'], format='%Y-%m-%d %H:%M:%S')
# result_h4m['Event Date'] = pd.to_datetime(result_h4m['Event Date'], format='%Y-%m-%d %H:%M:%S')

In [0]:
#Convertir a formato datetime, pero quitando la Zona horaria, y solo quedandonos hasta la parte de Segundos
datos_c4m['instant_date_t'] = pd.to_datetime(datos_c4m['instant_date_t'], errors='coerce', utc=False).dt.tz_localize(None)

#Convertir a formato datetime
result_h4m['Event Date'] = pd.to_datetime(result_h4m['Event Date'])

#### 4.2 Ver los resultados al cambiar de formato a datetime

In [0]:
datos_c4m['instant_date_t'].head()

In [0]:
result_h4m['Event Date'].head()

#### 4.3 Realizar el join especificando las columnas clave en cada DataFrame

In [0]:
# Realizar el join especificando las columnas clave en cada DataFrame
resultado_c4m_h4m = pd.merge(datos_c4m, result_h4m, left_on=['eq_id', 'instant_date_t', 'nombre'], right_on=['id_equipo', 'Event Date', 'Equipment'], how='left')

# resultado_join contendrá el DataFrame resultante después del join
resultado_c4m_h4m.head()

In [0]:
resultado_c4m_h4m_filtrado = resultado_c4m_h4m[resultado_c4m_h4m['Event Date'].notnull()]

In [0]:
resultado_c4m_h4m_filtrado[["eq_id","nombre_x","instant_date_t","Event Date"]].head()

In [0]:
resultado_c4m_h4m_filtrado.shape

In [0]:
resultado_c4m_h4m_filtrado['eq_id'].unique()


## 5. Guardamos la Tabla Consolidada C4M-H4M en el BlobStorage como csv  

In [0]:
#Librerias necesarias para usar el Azure DataLake y DataBrinks
import pandas as pd
from azure.storage.blob import BlobServiceClient

# 1.Guardamos el df convertido en csv en la variable csv_data
datos_total_c4m_h4m = resultado_c4m_h4m_filtrado.to_csv(index=False)

# 2. Obtener conection Azure DataLake,interfaz de Azure(Claves de acceso: Key1) (Debes comentar esta variable Si NO deja hacer COMMIT DEL 
# CODIGO EN GIT,GITHUB)
#connection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# 3. Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 4. Identificamos el nombre del contenedor(container_name) y nombre del archivo(definir blob_name) en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_consolidado/"
blob_name = "datos_raw_consolidado.csv"

# 5. Guardar el archivo CSV al Blob Storage
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
blob_client.upload_blob(datos_total_c4m_h4m)

In [0]:
import numpy as np